In [26]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

import pyrr
import itertools

# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
mockname = 'test_kiwi_2'
voxel_size = 0.1
downsample = 0.2

## Tree and leaves segmentation

Now we create the module to segmentate trees. This will be tuned acordingly for each data set, so below module only works for this particular data set.

In [28]:
def segtree(df, leaves, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 2, 2
    # dx, dy = 5, 5

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees


We segmentate the trees below,

In [29]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
N = len(df)

In [30]:

def voxel_subsampling(voxel_size, POINTS):

    nb_vox = np.ceil((np.max(POINTS, axis=0) - np.min(POINTS, axis=0))/voxel_size)
    ni, nj, nk = nb_vox
    print('min point:', np.min(POINTS, axis=0))
    print('max point:', np.max(POINTS, axis=0))
    print('Number of voxels: i:%d, j:%d, k:%d --> Total: %d' %(ni, nj, nk, np.product(nb_vox)))

    non_empty_voxel_keys, inverse, nb_pts_per_voxel = np.unique(((POINTS - np.min(POINTS, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted = np.argsort(inverse)
    print('Number of non-empty voxels: %d' %(len(non_empty_voxel_keys)))

    voxel_grid={}
    voxel_grid_ptsidx = {}
    grid_barycenter,grid_candidate_center = [], []
    last_seen=0

    for idx, vox in enumerate(non_empty_voxel_keys):

        idxs_per_vox = idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]
        voxel_grid[tuple(vox)] = POINTS[idxs_per_vox]
        voxel_grid_ptsidx[tuple(vox)] = idxs_per_vox

        # grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))

        idx_grid_candidate_center = np.linalg.norm(voxel_grid[tuple(vox)] - np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()
        grid_candidate_center.append(voxel_grid_ptsidx[tuple(vox)][idx_grid_candidate_center])

        last_seen+=nb_pts_per_voxel[idx]

    # print('Downsampling percentage: %.1f %%' %(100 * len(grid_candidate_center) / len(POINTS)))
    # minpoint = np.min(POINTS, axis=0)

    return list(grid_candidate_center) #, minpoint

In [31]:
def random_downsample(N, downsample):

    resdir = os.path.join(_data, mockname, 'random_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        idx = np.load(outdir)

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, N, int(N * downsample))
        # inds = np.zeros(N, dtype=bool)
        # inds[idx] = True

        np.save(outdir, idx)

    return idx

### Implement and keep Downsamplied points

In [32]:

if downsample is not None:
    inds = random_downsample(N, downsample)
else:
    inds = voxel_subsampling(voxel_size, df[['x', 'y', 'z']].to_numpy())

print('Downsampling percentage: %.1f %%' %(100 *  len(inds) / len(df['x'])))

df = df.iloc[inds]
POINTS = df[['x', 'y', 'z']].to_numpy()
SENSORS = df[['sx', 'sy', 'sz']].to_numpy()

# Compute lower point
minpoint = np.min(POINTS, axis=0)
print('minpoint:', minpoint)


inds not been created yet for donwnsample of 0.200
Downsampling percentage: 20.0 %
minpoint: [-5.00000238e+00 -5.00000095e+00 -1.43051147e-06]


### leave and tree segmentation

In [33]:
# extract leaves. Boolean array output
leaves = loads.extract_leaves(df, show=False)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df, leaves, show=False)

### Second downsampling: keep only points that colide with Plant Region

In [34]:
inPR = (leaves) & (trees['tree_0'])
minBB, maxBB = np.min(POINTS[inPR.values], axis=0), np.max(POINTS[inPR.values], axis=0)

# Make sure Plant Region min & max points are multiples of voxel size
# to match first voxelization where we implemented the downsampling
minpointPR = minpoint + np.floor(np.abs(minpoint - minBB)/voxel_size) * voxel_size
maxpointPR = minpoint + np.ceil(np.abs(minpoint - maxBB)/voxel_size) * voxel_size
boxPR = pyrr.aabb.create_from_bounds(minpointPR, maxpointPR)

lines = np.stack((POINTS, SENSORS), axis=1)
f = lambda line: pyrr.geometric_tests.ray_intersect_aabb(pyrr.ray.create_from_line(line), boxPR) is not None
res = np.array(list(map(f, lines)))

In [35]:
POINTS, SENSORS = POINTS[res], SENSORS[res]

leaves = leaves[res]

for key, val in trees.items():
    trees[key] = val[res]

In [36]:
# save indexes of voxel-based downsample

idxs = np.array(inds)[res]

if downsample is not None:
    dirname = 'random_%s' %(str(downsample))
    resdir = os.path.join(_data, mockname, dirname, 'lad_%s' %(str(voxel_size)))
else:
    dirname = 'voxel'
    resdir = os.path.join(_data, mockname, dirname, 'lad_%s' %(str(voxel_size)))

if not os.path.exists(resdir): os.makedirs(resdir)
outdir = os.path.join(resdir, 'inds.npy')
np.save(outdir, idxs)

## Ray tracing

In [39]:
sample = None

inPR = (leaves) & (trees['tree_0'])

# resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))
# if not os.path.exists(resdir):
    # os.makedirs(resdir)
if sample is not None:
    print('# iter...', len(POINTS[::sample]))
    m3s = rayt.main2(POINTS[::sample], SENSORS[::sample], POINTS[inPR], voxel_size, resdir, 'tree_0', (minpointPR, maxpointPR), show=True)
else:
    print('# iter...', len(POINTS))
    m3s = rayt.main2(POINTS, SENSORS, POINTS[inPR], voxel_size, resdir, 'tree_0', (minpointPR, maxpointPR), show=False)

# iter... 63112


63112it [12:00, 87.59it/s] 

tot vox: 	 11935
voxels hitted: 	 7459
Percentage of voxels hitted by beam: 0.62
voxels hitted (OLD): 	 0
Percentage of voxels hitted by beam (OLD): 0.00


In [40]:
inPR = (leaves) & (trees['tree_0'])
# resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))
NI, N0, NP0, NP = lad.get_attributes_per_k(POINTS[inPR], voxel_size, (minpointPR, maxpointPR), 'tree_0', kval=2, resdir=resdir, showall=True)

Number of voxels with points: 2437
-----------------
New attribute 1:
	 from fraction of incidences with attribute 1: 2020.14
	 from normal counts of voxels with attribute 1: 861.00
New attribute 2
	 from incidences with attribute 1: -444.14
	 from normal counts of voxels with attribute 2: 7459.00


## LIA

In [46]:
def best_fit_lia(mockname):

    df = loads.npy2pandas(mockname)
    # extract leaves. Boolean array output
    leaves = loads.extract_leaves(df, show=False)
    # extract trees. Dictionary with boolean arrays output
    trees = segtree(df, leaves, show=False)

    for key, val in trees.items():

        keep = (val) & (leaves) # take the LPC per tree
        points = df[['x', 'y', 'z']].to_numpy()[keep]

        res = lia.bestfit_pars_la(points, mockname, treename=key)
        lia.best_fit_pars_plot(res, key, mockname)

In [45]:
def get_lia(mockname):

    df = loads.npy2pandas(mockname)

    # extract leaves. Boolean array output
    leaves = loads.extract_leaves(df, show=False)
    # extract trees. Dictionary with boolean arrays output
    trees = segtree(df, leaves, show=False)


    # load bestfit results
    for key, val in trees.items():

        keep = (val) & (leaves)
        points = df[['x', 'y', 'z']].to_numpy()[keep]
    
        bestfit_file = os.path.join(_data, mockname, 'lia', 'bestfit_%s.npy' %(key))
        res = np.load(bestfit_file, allow_pickle=True)
        res = res.tolist()

        text = 'leaf area=%.2f \n %s=%.4f \n %s=%.4f \n %s=%.4f ' %(res['leafsize'], 'voxel_size_w', res['voxel_size_w_bestfit'],'kd3_sr', res['kd3_sr_bestfit'],'max_nn', res['max_nn_bestfit'])
        print(text)

        chi2 = lia.leaf_angle(points, mockname, key, res['voxel_size_w_bestfit'], 
                                res['kd3_sr_bestfit'], res['max_nn_bestfit'], save=True,
                                    savefig=True, text=text, voxel_size_h=0.1)

        # save indexes from main df
        inds = np.where((val) & (leaves))
        np.save(os.path.join(_data, mockname, 'lia', 'inds.npy'), inds[0])


In [47]:
best_fit_lia(mockname)

voxel_size_w 0.0001 DONE...
voxel_size_w 0.001 DONE...
voxel_size_w 0.01 DONE...
voxel_size_w 0.1 DONE...
voxel_size_w 1 DONE...
voxel_size_w BESTFIT:	 0.01
kd3_sr 0.001 DONE...
kd3_sr 0.01 DONE...
kd3_sr 0.1 DONE...
kd3_sr 1.0 DONE...
kd3_sr BESTFIT:	 0.1
max_nn 3 DONE...
max_nn 5 DONE...
max_nn 10 DONE...
max_nn 20 DONE...
max_nn 50 DONE...
max_nn 100 DONE...
max_nn BESTFIT:	 5


In [48]:
get_lia(mockname)

leaf area=0.01 
 voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=5.0000 


In [58]:
inds_file = os.path.join(resdir, 'inds.npy')
inds = np.load(inds_file)
print(inds)

[ 49026 141233 162125 ... 271647 121208 207455]


In [59]:
inds_lia = np.load(os.path.join(_data, mockname, 'lia', 'inds.npy'))
print(inds_lia)

[   105    106    137 ... 374846 374847 374848]


In [56]:
def runall(pointsPR, sensorsPR, voxel_size, tree, N, PRbounds, resdir, kbins=None):

    # resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))
    inds_file = os.path.join(resdir, 'inds.npy')
    inds = np.load(inds_file)
    print(inds)

    # resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))

    inds_lia = np.load(os.path.join(_data, mockname, 'lia', 'inds.npy'))

    isfigures = os.path.join(resdir, 'figures')
    if not os.path.exists(isfigures):
        os.makedirs(isfigures)

    attributes2_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(tree, str(voxel_size)))
    if os.path.isfile(attributes2_file):
        m3b = np.load(attributes2_file)

    print('voxel_size:', voxel_size)

    m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, tree, PRbounds)

    # _,_,_, m3scount = lad.density_counts(pointsPR, voxel_size)

    # get in down sample boolean array for LPC size
    mask1 = np.zeros(N, bool)
    mask2 = mask1.copy()

    mask1[inds] = True
    mask2[inds_lia] = True

    lias, ws = lad.downsample_lia(mockname, tree, np.where(mask1[mask2])[0])
    voxk = lad.get_voxk(pointsPR, PRbounds, voxel_size)
    bia = lad.get_bia(pointsPR, sensorsPR)
    meshfile = lad.get_meshfile(mockname)

    print('DEBUG')
    print(len(lias), len(ws), len(voxk))

    figext = '%s_%s' %(tree, str(voxel_size))
    # figext = None
    
    alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, meshfile, figext=figext, 
                            klia=False, use_true_lia=True)

    kmax = m3b.shape[2]
    if kbins is None:
        kbins = int(kmax/15)
    print(kbins)

    # Attribute 2 counts per voxel
    # outdir_count = os.path.join(resdir, 'm3count_%s_%s.npy' %(tree, str(voxel_size)))
    
    lads_mid, clai = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6], PRbounds, tree, resdir)
    lads_0, _ = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6]*0+1, PRbounds, tree, resdir)
    # lads_mid_old, _ = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6], PRbounds, tree, resdir, oldlad=True)
    # lads_mid_old = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], alpha2=1)
    lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax, PRbounds)

    # lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0, 'Correction Weights':lads_mid_w}#, 'Correction counts':lads_mid_counts}
    lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0}
    # clai = lad.get_clai(m3att, alphas_k)
    attributes_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(tree, str(voxel_size)))
    if os.path.isfile(attributes_file):
        RT = 'Y'
    else:
        RT = 'N'
        
    text = {'tree':tree, 'VS':voxel_size, 'RT':RT, 'CLAI':np.round(clai, 3)}
    txt = []
    for key, val in text.items():
        txt.append('%s=%s \n' %(key, str(val)))
    text = (' ').join(txt)

    savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
    figures.plot_lads(lads, text, savefig=savefig)


In [57]:
inPR = (leaves) & (trees['tree_0'])
runall(POINTS[inPR], SENSORS[inPR], voxel_size, 'tree_0', N, (minpointPR, maxpointPR), resdir, kbins=1)

/Users/omar/projects/planttech/data/test_kiwi_2/random_0.2/lad_0.1/inds.npy <class 'numpy.ndarray'>
voxel_size: 0.1
max --> [30, 34, 10]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (31, 35, 11)
ray tracker voxel dimensions: 	 (31, 35, 11)
Number of voxels ocupied by points cloud: 	 2437
Number of voxels ocupied by beam points cloud: 	 7459
Total number of voxels in plant regions: 	 11935
Number of voxels with attribute 1: 	 2437
Number of voxels with attribute 2: 	 7459
Number of voxels with attribute 3: 	 2039
DEBUG
30013 30013 33145


IndexError: boolean index did not match indexed array along dimension 0; dimension is 30013 but corresponding boolean dimension is 33145